In [2]:
!wget https://s3.amazonaws.com/drivendata-prod/data/66/public/training_set_features.csv
!wget https://s3.amazonaws.com/drivendata-prod/data/66/public/submission_format.csv
!wget https://s3.amazonaws.com/drivendata-prod/data/66/public/training_set_labels.csv
!wget https://s3.amazonaws.com/drivendata-prod/data/66/public/test_set_features.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


## Importing the needed libraries

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import  f1_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder



%matplotlib inline


## Importing data

In [6]:
path_train_X='./training_set_features.csv'
path_train_Y='./training_set_labels.csv'
path_test_X='./test_set_features.csv'

In [7]:
train_set_X = pd.read_csv(path_train_X)
print(train_set_X.head())

   respondent_id  h1n1_concern  h1n1_knowledge  behavioral_antiviral_meds  \
0              0           1.0             0.0                        0.0   
1              1           3.0             2.0                        0.0   
2              2           1.0             1.0                        0.0   
3              3           1.0             1.0                        0.0   
4              4           2.0             1.0                        0.0   

   behavioral_avoidance  behavioral_face_mask  behavioral_wash_hands  \
0                   0.0                   0.0                    0.0   
1                   1.0                   0.0                    1.0   
2                   1.0                   0.0                    0.0   
3                   1.0                   0.0                    1.0   
4                   1.0                   0.0                    1.0   

   behavioral_large_gatherings  behavioral_outside_home  \
0                          0.0               

In [8]:
train_set_Y = pd.read_csv(path_train_Y)
print(train_set_Y.head())
print(train_set_Y.shape)

   respondent_id  h1n1_vaccine  seasonal_vaccine
0              0             0                 0
1              1             0                 1
2              2             0                 0
3              3             0                 1
4              4             0                 0
(26707, 3)


In [9]:
test_set_X = pd.read_csv(path_test_X)
test_set_X.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


In [10]:
train_set_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 36 columns):
respondent_id                  26707 non-null int64
h1n1_concern                   26615 non-null float64
h1n1_knowledge                 26591 non-null float64
behavioral_antiviral_meds      26636 non-null float64
behavioral_avoidance           26499 non-null float64
behavioral_face_mask           26688 non-null float64
behavioral_wash_hands          26665 non-null float64
behavioral_large_gatherings    26620 non-null float64
behavioral_outside_home        26625 non-null float64
behavioral_touch_face          26579 non-null float64
doctor_recc_h1n1               24547 non-null float64
doctor_recc_seasonal           24547 non-null float64
chronic_med_condition          25736 non-null float64
child_under_6_months           25887 non-null float64
health_worker                  25903 non-null float64
health_insurance               14433 non-null float64
opinion_h1n1_vacc_effective  

In [11]:
test_set_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26708 entries, 0 to 26707
Data columns (total 36 columns):
respondent_id                  26708 non-null int64
h1n1_concern                   26623 non-null float64
h1n1_knowledge                 26586 non-null float64
behavioral_antiviral_meds      26629 non-null float64
behavioral_avoidance           26495 non-null float64
behavioral_face_mask           26689 non-null float64
behavioral_wash_hands          26668 non-null float64
behavioral_large_gatherings    26636 non-null float64
behavioral_outside_home        26626 non-null float64
behavioral_touch_face          26580 non-null float64
doctor_recc_h1n1               24548 non-null float64
doctor_recc_seasonal           24548 non-null float64
chronic_med_condition          25776 non-null float64
child_under_6_months           25895 non-null float64
health_worker                  25919 non-null float64
health_insurance               14480 non-null float64
opinion_h1n1_vacc_effective  

## Merging the two datasets

In [12]:
train_set = pd.DataFrame(pd.concat((train_set_X,train_set_Y[['h1n1_vaccine' , 'seasonal_vaccine']]),axis=1))
train_set['respondent_id'] = train_set['respondent_id'].astype(str)
test_set_X['respondent_id'] = test_set_X['respondent_id'].astype(str)
print(train_set.head())


  respondent_id  h1n1_concern  h1n1_knowledge  behavioral_antiviral_meds  \
0             0           1.0             0.0                        0.0   
1             1           3.0             2.0                        0.0   
2             2           1.0             1.0                        0.0   
3             3           1.0             1.0                        0.0   
4             4           2.0             1.0                        0.0   

   behavioral_avoidance  behavioral_face_mask  behavioral_wash_hands  \
0                   0.0                   0.0                    0.0   
1                   1.0                   0.0                    1.0   
2                   1.0                   0.0                    0.0   
3                   1.0                   0.0                    1.0   
4                   1.0                   0.0                    1.0   

   behavioral_large_gatherings  behavioral_outside_home  \
0                          0.0                     

## Dealing with null values:

In [25]:
train_set.isnull().sum()

respondent_id                      0
h1n1_concern                      92
h1n1_knowledge                   116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_h1n1_vacc_effective      391
opinion_h1n1_risk                388
opinion_h1n1_sick_from_vacc      395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
m

### Deleting columns with too mant missing values:

In [32]:
sums = train_set.isnull().sum()
columns = list(sums[sums > 0.4 * train_set.shape[0]].keys())
columns

['health_insurance', 'employment_industry', 'employment_occupation']

In [73]:
df = train_set.drop(columns=columns)
df.isnull().sum()[:5]

respondent_id                  0
h1n1_concern                  92
h1n1_knowledge               116
behavioral_antiviral_meds     71
behavioral_avoidance         208
dtype: int64

In [74]:
df.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,seasonal_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,0,1
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,0,1
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,0,0


### Imputing the others with the most frequent value:

In [75]:
impt = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
df = pd.DataFrame(impt.fit_transform(df),columns=df.columns)
df.isnull().sum().sum()

0

In [76]:
df.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,seasonal_vaccine
0,0,1,0,0,0,0,0,0,1,1,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0,0,0,0
1,1,3,2,0,1,0,1,0,1,1,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0,0,0,1
2,2,1,1,0,1,0,0,0,0,0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2,0,0,0
3,3,1,1,0,1,0,1,1,0,0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0,0,0,1
4,4,2,1,0,1,0,1,1,0,1,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1,0,0,0


## using LabelEncoder to label the columns:

In [77]:
le = LabelEncoder()
classes= {}
for col in df.columns:
    df[col] = le.fit_transform(df[col])
    classes[col] = le.classes_
classes    

{'respondent_id': array(['0', '1', '10', ..., '9997', '9998', '9999'], dtype=object),
 'h1n1_concern': array([0.0, 1.0, 2.0, 3.0], dtype=object),
 'h1n1_knowledge': array([0.0, 1.0, 2.0], dtype=object),
 'behavioral_antiviral_meds': array([0.0, 1.0], dtype=object),
 'behavioral_avoidance': array([0.0, 1.0], dtype=object),
 'behavioral_face_mask': array([0.0, 1.0], dtype=object),
 'behavioral_wash_hands': array([0.0, 1.0], dtype=object),
 'behavioral_large_gatherings': array([0.0, 1.0], dtype=object),
 'behavioral_outside_home': array([0.0, 1.0], dtype=object),
 'behavioral_touch_face': array([0.0, 1.0], dtype=object),
 'doctor_recc_h1n1': array([0.0, 1.0], dtype=object),
 'doctor_recc_seasonal': array([0.0, 1.0], dtype=object),
 'chronic_med_condition': array([0.0, 1.0], dtype=object),
 'child_under_6_months': array([0.0, 1.0], dtype=object),
 'health_worker': array([0.0, 1.0], dtype=object),
 'opinion_h1n1_vacc_effective': array([1.0, 2.0, 3.0, 4.0, 5.0], dtype=object),
 'opinion_h1n1

## Extracting non-binary features:

In [78]:
columns_categorical=[]
for col in df.columns[1:]:
    if len(df[col].unique()) > 2:
        columns_categorical.append(col)
columns_categorical

['h1n1_concern',
 'h1n1_knowledge',
 'opinion_h1n1_vacc_effective',
 'opinion_h1n1_risk',
 'opinion_h1n1_sick_from_vacc',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc',
 'age_group',
 'education',
 'race',
 'income_poverty',
 'employment_status',
 'hhs_geo_region',
 'census_msa',
 'household_adults',
 'household_children']

## OHE for the non-binary columns

In [84]:
XY = pd.get_dummies(df,columns=columns_categorical)
XY.head()

,respondent_id,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,...,census_msa_1,census_msa_2,household_adults_0,household_adults_1,household_adults_2,household_adults_3,household_children_0,household_children_1,household_children_2,household_children_3
0,0,0,0,0,0,0,1,1,0,0,...,0,1,1,0,0,0,1,0,0,0
1,1,0,1,0,1,0,1,1,0,0,...,0,0,1,0,0,0,1,0,0,0
2,11112,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,18930,0,1,0,1,1,0,0,0,1,...,1,0,1,0,0,0,1,0,0,0
4,20041,0,1,0,1,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
